# Ingest tickers symbols

In [0]:
%run "../utils/mount_configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType, BooleanType

tickers_symbol_schema = StructType([
  StructField("count", LongType(), True),
  StructField("next_url", StringType(), True),
  StructField("request_id", StringType(), True),
  StructField("results", ArrayType(StructType([
    StructField("ticker", StringType(), True),
    StructField("name", StringType(), True),
    StructField("market", StringType(), True),
    StructField("locale", StringType(), True),
    StructField("primary_exchange", StringType(), True),
    StructField("type", StringType(), True),
    StructField("active", BooleanType(), True),
    StructField("currency_name", StringType(), True),
    StructField("cik", StringType(), True),
    StructField("composite_figi", StringType(), True),
    StructField("share_class_figi", StringType(), True),
    StructField("last_updated_utc", StringType(), True)
  ])), True)
])

In [0]:
tickers_symbol_df = spark.read \
    .schema(tickers_symbol_schema) \
    .json(f"{raw_folder_path}/tickers_symbol/all_tickers_symbol*.json")

# display(tickers_symbol_df)

In [0]:
from pyspark.sql.functions import explode, col

tickers_symbol_final_df = tickers_symbol_df \
  .select(explode(col("results")) \
  .alias("result")) \
  .select("result.*") \
  .withColumnRenamed("name", "company_name") \
  .select(col("ticker"), col("company_name"))  
  

# display(tickers_symbol_final_df)

In [0]:
tickers_symbol_final_df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("engineering_processed.tickers_symbol")

In [0]:
%sql
SELECT * FROM engineering_processed.tickers_symbol;